<a href="https://colab.research.google.com/github/dmswl0707/Siamese-Networks/blob/main/model__train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class siamese_network(nn.Module):
    def __init__(self):
        super(siamese_network, self).__init__()
        self.cnn = nn.Sequential(
            nn.ReflectionPad2d(1),   #
            nn.Conv2d(1, 4, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(4),

            nn.ReflectionPad2d(1),
            nn.Conv2d(4, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),

            nn.ReflectionPad2d(1),
            nn.Conv2d(8, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8)  
        )
        self.fc = nn.Sequential(
            nn.Linear(),
            nn.ReLU(inplace=True),

            nn.Linear(),
            nn.ReLU(inplace=True)

            nn.Linear()
        )

    def forward_once(self, x):
        output = self.cnn(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2


In [ ]:
class Contrastive_loss(torch.nn.Module): 
    def __init__(self, margin=2.0):
        super(contrastive_loss, self).__init__()
        self.magin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1- label) * torch.pow(euclidean_distance, 2)
                                       (label) * torch.pow(torch.clamp(self.margin - euclidean_distanxe, min=0.2), 2))
        
        return loss_contrastive                               

In [ ]:
train_dataloader = DataLoader()

net = siamese_network().cuda()
criterion = Contrastive_loss()
optimizer = optim.Adam(net.parameters(), lr=0.0005)

In [ ]:
counter = []
loss_history = []
iteration_number = 0

for epoch in range(0, num_epochs):
    for i, data in enumerate(train_dataloader, 0):
        img0, img1, label = data
        img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()
        optimizer.zero_grad()
        output1, output2 = net(img0, img1)
        loss_contrastive = criterion(ouput1, output2, label)
        loss_contrastive.backward()
        optimizer.step()
        if i % 10 == 0:
            print("epoch number {}\n current loss {}\n".format(epoch, loss_contrastive.item()))
            iteration_number += 10
            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())


show_plot(counter, loss_history)


In [ ]:
dataiter = iter(test_dataloader)
x0,_,_ = next(dataiter)

for i in range(10):
    _,x1,label2 = next(dataiter)
    concatenated = torch.cat((x0,x1),0)
    
    output1,output2 = net(Variable(x0).cuda(),Variable(x1).cuda())
    euclidean_distance = F.pairwise_distance(output1, output2)
    imshow(torchvision.utils.make_grid(concatenated),'Dissimilarity: {:.2f}'.format(euclidean_distance.item()))